In [14]:
import csv
from collections import Counter
import math
import os
import random

# Укажите путь к вашему файлу 'cities.csv'
csv_file_path = 'C:/Users/артем/Desktop/CreatingPro/Лабораторные/DZ_2024_05_01/cities.csv'

# Открытие CSV-файла и загрузка данных
with open(csv_file_path, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Вывести в консоль список всех регионов (только уникальные значения) и отсортировать по алфавиту
regions = sorted(set(row['Регион'] for row in data))
print("Список регионов:")
print("\n".join(regions))
print()

# Создать csv-файл с городами, в названии которых есть буква 'ы'
cities_with_y = [row for row in data if 'ы' in row['Город']]
with open('cities_with_y.csv', 'w', newline='', encoding='utf-8') as file:
    fieldnames = data[0].keys()
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(cities_with_y)

# Создать csv-файл, данные отсортированы по названию округа, региона, города
sorted_data = sorted(data, key=lambda x: (x['Федеральный округ'], x['Регион'], x['Город']))
with open('cities_sorted.csv', 'w', newline='', encoding='utf-8') as file:
    fieldnames = data[0].keys()
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(sorted_data)
if not os.path.exists('temp'):
    os.makedirs('temp')

# Создать N csv-файлов с данными по каждому региону без столбца 'Регион'
for region in set(row['Регион'] for row in data):
    regional_data = [row for row in data if row['Регион'] == region]
    regional_data = [{k: v for k, v in row.items() if k != 'Регион'} for row in regional_data]
    filename = f"cities - {region}.csv"
    with open(os.path.join('temp', filename), 'w', newline='', encoding='utf-8') as file:
        fieldnames = regional_data[0].keys()
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(regional_data)

# Вывести в консоль список городов, название которых содержит до 4-х букв включительно и отсортированные в обратном алфавитном порядке
short_cities = [row['Город'] for row in data if len(row['Город']) <= 4]
short_cities.sort(reverse=True)
print("Города с названием до 4-х букв включительно в обратном алфавитном порядке:")
print("\n".join(short_cities))
print()

# Вывести в консоль список регионов и количество городов в каждом регионе
region_counts = Counter(row['Регион'] for row in data)
print("Количество городов в каждом регионе:")
for region, count in sorted(region_counts.items()):
    print(f"{region}: {count}")
print()

# Вывести в консоль список федеральных округов с количеством регионов и количеством городов
district_counts = Counter(row['Федеральный округ'] for row in data)
district_region_counts = {district: len(set(row['Регион'] for row in data if row['Федеральный округ'] == district)) for district in district_counts}
print("Количество регионов и городов в каждом федеральном округе:")
for district, region_count in sorted(district_region_counts.items()):
    city_count = district_counts[district]
    print(f"{district}: {region_count} регионов, {city_count} городов")
print()

# Вывести в консоль буквы алфавита по убыванию и рядом количество городов, начинающихся на каждую букву
city_start_counts = Counter(row['Город'][0].lower() for row in data if row['Город'] and row['Город'][0].isalpha())
print("Количество городов, начинающихся на каждую букву:")
for letter, count in sorted(city_start_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{letter.upper()}: {count}")
print()

# Вывести в консоль числа по возрастанию и рядом количество городов, состоящих из указанного количества букв
city_length_counts = Counter(len(row['Город']) for row in data)
print("Количество городов с указанным количеством букв:")
for length, count in sorted(city_length_counts.items()):
    print(f"{length}: {count}")
print()

# Вывести в консоль список ТОП-10 городов с самыми длинными названиями
longest_cities = sorted(data, key=lambda x: len(x['Город']), reverse=True)[:10]
print("ТОП-10 городов с самыми длинными названиями:")
for city in longest_cities:
    print(city['Город'])
print()

# Вывести в консоль список ТОП-10 городов, в названии которых больше всего согласных букв
def count_consonants(word):
    consonants = 'бвгджзклмнпрстфхцчшщ'
    return sum(letter.lower() in consonants for letter in word if letter.isalpha())

cities_by_consonants = sorted(data, key=lambda x: count_consonants(x['Город']), reverse=True)[:10]
print("ТОП-10 городов с наибольшим количеством согласных букв:")
for city in cities_by_consonants:
    print(city['Город'])
print()

# Вывести в консоль список ТОП-10 букв, которые используются чаще всего в названии городов
all_letters = ''.join(row['Город'].lower() for row in data if row['Город'].isalpha())
letter_counts = Counter(all_letters)
top_letters = sorted(letter_counts.items(), key=lambda x: x[1], reverse=True)[:10]
print("ТОП-10 букв, которые используются чаще всего в названии городов:")
for letter, count in top_letters:
    print(f"{letter.upper()}: {count}")
print()

# Вывести в консоль расстояние между случайными двумя городами
city1 = random.choice(data)
city2 = random.choice(data)

lat1, lon1 = float(city1['Широта']), float(city1['Долгота'])
lat2, lon2 = float(city2['Широта']), float(city2['Долгота'])

# Формула для расчета расстояния между двумя точками на сфере (approximation)
R = 6371  # Радиус Земли в километрах
phi1 = math.radians(lat1)
phi2 = math.radians(lat2)
delta_phi = math.radians(lat2 - lat1)
delta_lambda = math.radians(lon2 - lon1)

a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
distance = R * c

print(f"Расстояние между {city1['Город']} и {city2['Город']}: {distance:.2f} км")
print()

Список регионов:
Адыгея
Алтай
Алтайский
Амурская
Архангельская
Башкортостан
Белгородская
Брянская
Бурятия
Волгоградская
Вологодская
Воронежская
Дагестан
Еврейская
Забайкальский
Ивановская
Ингушетия
Иркутская
Кабардино-Балкарская
Калининградская
Калужская
Камчатский
Карачаево-Черкесская
Карелия
Кемеровская
Кировская
Коми
Костромская
Краснодарский
Красноярский
Крым
Курганская
Ленинградская
Липецкая
Магаданская
Марий Эл
Мордовия
Москва
Московская
Мурманская
Ненецкий
Нижегородская
Новгородская
Новосибирская
Омская
Оренбургская
Орловская
Пензенская
Пермский
Приморский
Псковская
Ростовская
Рязанская
Самарская
Санкт-Петербург
Саратовская
Сахалинская
Свердловская
Севастополь
Северная Осетия - Алания
Смоленская
Ставропольский
Тамбовская
Татарстан
Тверская
Томская
Тульская
Тыва
Тюменская
Удмуртская
Ульяновская
Хабаровский
Хакасия
Ханты-Мансийский Автономный округ - Югра
Челябинская
Чеченская
Чувашская Республика
Чукотский
Ямало-Ненецкий
Ярославская

Города с названием до 4-х букв включительно в 